# OPE/OPL Experiments with Synthetic Bandit Data
---
This notebook provides an example of conducting OPE of several different evaluation policies with synthetic bandit feedback data.

In [1]:
from sklearn.linear_model import LogisticRegression

# import open bandit pipeline (obp)
import obp
from obp.dataset import (
    SyntheticBanditDataset,
    logistic_reward_function,
    linear_behavior_policy,
)
from obp.policy import IPWLearner
from obp.ope import (
    OffPolicyEvaluation, 
    RegressionModel,
    InverseProbabilityWeighting as IPS,
    DirectMethod as DM,
    DoublyRobust as DR,
)

In [2]:
# obp version
print(obp.__version__)

0.5.0


## (1) Generate Synthetic Data

`SyntheticBanditDataset` is an easy-to-use synthetic data generator class in the dataset module.

In [3]:
dataset = SyntheticBanditDataset(
    n_actions=10, # number of actions; |A|
    dim_context=5, # number of dimensions of context vector
    reward_function=logistic_reward_function, # mean reward function; q(x,a)
    behavior_policy_function=linear_behavior_policy, # behavior policy; \pi_b
    random_state=12345,
)

In [4]:
training_bandit_data = dataset.obtain_batch_bandit_feedback(n_rounds=10000)
test_bandit_data = dataset.obtain_batch_bandit_feedback(n_rounds=1000000)

In [5]:
training_bandit_data

{'n_rounds': 10000,
 'n_actions': 10,
 'context': array([[-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057],
        [ 1.39340583,  0.09290788,  0.28174615,  0.76902257,  1.24643474],
        [ 1.00718936, -1.29622111,  0.27499163,  0.22891288,  1.35291684],
        ...,
        [-1.27028221,  0.80914602, -0.45084222,  0.47179511,  1.89401115],
        [-0.68890924,  0.08857502, -0.56359347, -0.41135069,  0.65157486],
        [ 0.51204121,  0.65384817, -1.98849253, -2.14429131, -0.34186901]]),
 'action_context': array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]),
 'action': array([9, 2, 1, ..., 0, 2, 6]),
 'position': None,
 're

## (2) Train Bandit Policies (OPL)
`obp.policy.IPWLearner` can be a first choice (IPS=IPW)

$ \hat{\pi} \in \underset{\pi \in \Pi}{\operatorname{argmax}} \hat{V}_{I P S}\left(\pi ; \mathcal{D}_{t r}\right) $

In [6]:
ipw_learner = IPWLearner(
    n_actions=dataset.n_actions, # number of actions; |A|
    base_classifier=LogisticRegression(C=100, random_state=12345) # any sklearn classifier
)

In [7]:
# fit
ipw_learner.fit(
    context=training_bandit_data["context"], # context; x
    action=training_bandit_data["action"], # action; a
    reward=training_bandit_data["reward"], # reward; r
    pscore=training_bandit_data["pscore"], # propensity score; pi_b(a|x)
)

In [8]:
# predict (action dist = action distribution)
action_dist_ipw = ipw_learner.predict(
    context=test_bandit_data["context"], # context in the test data
)

In [9]:
action_dist_ipw[:, :, 0] # which action to take for each context 

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

## (3) Approximate the Ground-truth Policy Value
$V(\pi) \approx \frac{1}{|\mathcal{D}_{te}|} \sum_{i=1}^{|\mathcal{D}_{te}|} \mathbb{E}_{a \sim \pi(a|x_i)} [q(x_i, a)], \; \, where \; \, q(x,a) := \mathbb{E}_{r \sim p(r|x,a)} [r]$

In [10]:
policy_value_of_ipw = dataset.calc_ground_truth_policy_value(
    expected_reward=test_bandit_data["expected_reward"], # expected rewards; q(x,a)
    action_dist=action_dist_ipw, # action distribution of IPWLearner
)

In [11]:
# ground-truth policy value of `IPWLearner`
policy_value_of_ipw

0.7544002774485176

## (4) Off-Policy Evaluation (OPE)

### (4-1) obtain a reward estimator
`obp.ope.RegressionModel` simplifies the process of reward modeling

$q(x,a) = \mathbb{E} [r \mid x, a] \approx \hat{q}(x,a)$

In [12]:
regression_model = RegressionModel(
    n_actions=dataset.n_actions, # number of actions; |A|
    base_model=LogisticRegression(C=100, random_state=12345) # any sklearn classifier
)

In [13]:
estimated_rewards = regression_model.fit_predict(
    context=test_bandit_data["context"], # context; x
    action=test_bandit_data["action"], # action; a
    reward=test_bandit_data["reward"], # reward; r
    random_state=12345,
)

In [14]:
estimated_rewards[:, :, 0] # \hat{q}(x,a)

array([[0.83287191, 0.77609002, 0.86301082, ..., 0.80705541, 0.87089962,
        0.88661944],
       [0.24064512, 0.18060694, 0.28603087, ..., 0.21010776, 0.30020355,
        0.33212285],
       [0.92681158, 0.89803854, 0.94120582, ..., 0.91400784, 0.94487921,
        0.95208655],
       ...,
       [0.95590514, 0.93780227, 0.96479479, ..., 0.94790501, 0.96704598,
        0.97144249],
       [0.9041431 , 0.8677301 , 0.92262342, ..., 0.88785354, 0.92736823,
        0.93671186],
       [0.19985698, 0.14801146, 0.23998121, ..., 0.1733142 , 0.25267968,
        0.28157917]])

### (4-2) OPE
`obp.ope.OffPolicyEvaluation` simplifies the OPE process

$V(\pi_e) \approx \hat{V} (\pi_e; \mathcal{D}_b, \theta)$ using DM, IPS, and DR

In [15]:
ope = OffPolicyEvaluation(
    bandit_feedback=test_bandit_data, # test data
    ope_estimators=[
        IPS(estimator_name="IPS"), 
        DM(estimator_name="DM"), 
        DR(estimator_name="DR"), # used estimators
    ]
)

In [16]:
estimated_policy_value = ope.estimate_policy_values(
    action_dist=action_dist_ipw, # \pi_e(a|x)
    estimated_rewards_by_reg_model=estimated_rewards, # \hat{q}(x,a)
)

In [17]:
# OPE results given by the three estimators
estimated_policy_value

{'IPS': 0.7514146726298303, 'DM': 0.6496363798172724, 'DR': 0.7515054724486296}

## (5) Evaluation of OPE
Now, let's evaluate the OPE performance (estimation accuracy) of the three estimators

$SE (\hat{V}; \mathcal{D}_b) := \left( V(\pi_e) - \hat{V} (\pi_e; \mathcal{D}_b, \theta) \right)^2$,     (squared error of $\hat{V}$)

In [18]:
squared_errors = ope.evaluate_performance_of_estimators(
    ground_truth_policy_value=policy_value_of_ipw, # V(\pi_e)
    action_dist=action_dist_ipw, # \pi_e(a|x)
    estimated_rewards_by_reg_model=estimated_rewards, # \hat{q}(x,a)
    metric="se", # squared error
)

In [19]:
squared_errors # DR is the most accurate

{'IPS': 8.913836133368584e-06,
 'DM': 0.010975474246890016,
 'DR': 8.379895987376119e-06}

We can iterate the above process several times and calculate the following MSE

$MSE (\hat{V}) := T^{-1} \sum_{t=1}^T SE (\hat{V}; \mathcal{D}_b^{(t)}) $

where $\mathcal{D}_b^{(t)}$ is the synthetic data in the $t$-th iteration